# The DAG of Julia packages

## Problem statement

In this tutorial, we show how LG in conjunction with other utility packages can be used for extracting the most recent directed acyclic graph (DAG) of the Julia package system. This information can be used for interactive data visualization with [D3](https://d3js.org/) like in the following links:

- **The DAG of Julia packages:** https://juliohm.github.io/science/DAG-of-Julia-packages
- **Where are the Julians?** https://juliohm.github.io/science/where-are-the-julians

All the packages used in this notebook can be installed with:

In [1]:
for dep in ["HTTP","JSON","GitHub","LightGraphs","ProgressMeter"]
    Pkg.add(dep)
end

INFO: Nothing to be done
INFO: Nothing to be done
INFO: Nothing to be done
INFO: Nothing to be done
INFO: Nothing to be done


In order to be able to query information from GitHub without be misinterpreted as a malicious robot, you need to [create a personal token](https://github.com/settings/tokens) in your GitHub settings. Since this token is private, we ask you to save it as an environment variable in your operating system (e.g. set `GITHUB_AUTH` in your `.bashrc` file). This variable will be read in Julia and used for authentication as follows:

In [2]:
using HTTP
using JSON
using GitHub
using LightGraphs
using ProgressMeter

# authenticate with GitHub to increase query limits
mytoken = ENV["GITHUB_AUTH"]
myauth = GitHub.authenticate(mytoken)

GitHub.OAuth2(8cda0d**********************************)

After successful authentication, we are now ready to start coding. First, we extract the names of all registered packages in METADATA and assign to each of them a unique integer id:

In [3]:
# find all packages in METADATA
pkgs = readdir(Pkg.dir("METADATA"))
filterfunc = p -> isdir(joinpath(Pkg.dir("METADATA"), p)) && p ∉ [".git",".test"]
pkgs = filter(filterfunc, pkgs)

# assign each package an id
pkgdict = Dict{String,Int}()
for (i,pkg) in enumerate(pkgs)
  push!(pkgdict, pkg => i)
end
pkgdict

Dict{String,Int64} with 1435 entries:
  "Levenshtein"        => 691
  "ReadStat"           => 1093
  "Discretizers"       => 310
  "SchumakerSpline"    => 1161
  "FredData"           => 433
  "GaussQuadrature"    => 452
  "RecurrenceAnalysis" => 1099
  "MKLSparse"          => 808
  "AnsiColor"          => 18
  "ProximalOperators"  => 1029
  "Luxor"              => 743
  "RobustLeastSquares" => 1138
  "Temporal"           => 1296
  "Robotlib"           => 1136
  "PiecewiseLinearOpt" => 983
  "JLDArchives"        => 634
  "MatrixDepot"        => 769
  "CodeTools"          => 157
  "NumericSuffixes"    => 897
  "COBRA"              => 155
  "Crypto"             => 222
  "Mongo"              => 822
  "ROOT"               => 1146
  "MNIST"              => 814
  "RandomMatrices"     => 1075
  ⋮                    => ⋮

Using the ids, we can easily build the DAG of packages with LG:

In [4]:
# build DAG
DAG = DiGraph(length(pkgs))
@showprogress 1 "Building graph..." for pkg in pkgs
  children = Pkg.dependents(pkg)
  for c in children
    add_edge!(DAG, pkgdict[pkg], pkgdict[c])
  end
end

Building graph...100% Time: 0:03:53


We are interested in finding all the descendents of a package. In other words, we are interested in finding all packages that are influenced by a given package. In this context, we further want to save the level of dependency (or geodesic distance) from descendents to the package being queried. This is a straightforward operation in LG:

In [5]:
# find (indirect) descendents
descendents = []
for pkg in pkgs
  gdists = gdistances(DAG, pkgdict[pkg])
  desc = [Dict("id"=>pkgs[v], "level"=>gdists[v]) for v in find(gdists .> 0)]
  push!(descendents, desc)
end

For each package, we also want to save information about who has contributed to the project. This task is easy to implement with the awesome [GitHub.jl](https://github.com/JuliaWeb/GitHub.jl) API. However, some of the packages registered in METADATA are hosted on different websites such as gitlab, for which an API is missing. We simply skip them and ask authors to migrate their code to GitHub if possible:

In [6]:
# find contributors
pkgcontributors = []
hostnames = []
@showprogress 1 "Finding contributors..." for pkg in pkgs
  urlfile = joinpath(Pkg.dir("METADATA"), pkg, "url")
  url = readline(urlfile)
  m = match(r".*://([a-z.]*)/(.*)\.git.*", url)
  hostname = m[1]; reponame = m[2]
  if hostname == "github.com"
    users, _ = contributors(reponame, auth=myauth)
    usersdata = map(u -> (u["contributor"].login, u["contributions"]), users)
    pkgcontrib = [Dict("id"=>u, "contributions"=>c) for (u,c) in usersdata]
    push!(pkgcontributors, pkgcontrib)
    push!(hostnames, hostname)
  else
    push!(pkgcontributors, [])
    push!(hostnames, hostname)
  end
end

Finding contributors...100% Time: 0:11:51


We also extract the Julia version required in the last tag of a package. Both the lower and upper bounds are saved as well as a "cleaned" `major.minor` string for the lower bound, which is useful for data visualization:

In [7]:
# find required Julia version
juliaversion = []
for pkg in pkgs
  versiondir = joinpath(Pkg.dir("METADATA"), pkg, "versions")
  if isdir(versiondir)
    latestversion = readdir(versiondir)[end]
    reqfile = joinpath(versiondir, latestversion, "requires")
    reqs = Pkg.Reqs.parse(reqfile)
    if "julia" ∈ keys(reqs)
      vinterval = reqs["julia"].intervals[1]
      vmin = vinterval.lower
      vmax = vinterval.upper
      majorminor = "v$(vmin.major).$(vmin.minor)"
      push!(juliaversion, Dict("min"=>string(vinterval.lower),
                               "max"=>string(vinterval.upper),
                               "majorminor"=>majorminor))
    else
      push!(juliaversion, Dict("min"=>"NA", "max"=>"NA", "majorminor"=>"NA"))
    end
  else
    push!(juliaversion, Dict("min"=>"BOGUS", "max"=>"BOGUS", "majorminor"=>"BOGUS"))
  end
end

Finally, we save the data in a JSON file:

In [8]:
# construct JSON
nodes = [Dict("id"=>pkgs[v],
              "indegree"=>indegree(DAG,v),
              "outdegree"=>outdegree(DAG,v),
              "juliaversion"=>juliaversion[v],
              "descendents"=>descendents[v],
              "contributors"=>pkgcontributors[v]) for v in vertices(DAG)]

links = [Dict("source"=>pkgs[u], "target"=>pkgs[v]) for (u,v) in edges(DAG)]

data = Dict("nodes"=>nodes, "links"=>links)

# write to file
open("DAG-Julia-Pkgs.json", "w") do f
  JSON.print(f, data, 2)
end

## Where are the Julians?

Having extracted and saved the DAG of Julia packages, we take this opportunity to find out the Julians responsible for this amazing package system.

We use LG again to build this social network:

In [9]:
# find Julians on Github
julians = []
for pkgcontrib in pkgcontributors
    append!(julians, [julian["id"].value for julian in pkgcontrib])
end
julians = sort(unique(julians))

# assign each Julian an id
juliandict = Dict{String,Int}()
for (i,julian) in enumerate(julians)
  push!(juliandict, julian => i)
end
juliandict

Dict{String,Int64} with 1500 entries:
  "credentiality"    => 480
  "ZacCranko"        => 230
  "Snnappie"         => 199
  "benhamner"        => 376
  "lynyus"           => 951
  "iraikov"          => 752
  "nstiurca"         => 1094
  "pearlzli"         => 1130
  "GunnarFarneback"  => 85
  "njwilson23"       => 1086
  "gustafsson"       => 693
  "cgoldammer"       => 436
  "garrison"         => 656
  "lobingera"        => 938
  "randyzwitch"      => 1181
  "JonathanAnderson" => 116
  "madanim"          => 958
  "Armavica"         => 18
  "Matt5sean3"       => 141
  "slangangular"     => 1303
  "raphapr"          => 1185
  "kuldeepdhaka"     => 910
  "jdrugo"           => 787
  "J-Revell"         => 98
  "fserra"           => 644
  ⋮                  => ⋮

In [10]:
# build the social network
socialnet = Graph(length(julians))
contribdict = Dict{String,Int}()
for pkgcontrib in pkgcontributors
    ids = [julian["id"].value for julian in pkgcontrib]
    contribs = [julian["contributions"] for julian in pkgcontrib]
    for i=1:length(ids)
        contribdict[ids[i]] = get(contribdict, ids[i], 0) + contribs[i]
    end
    for i=1:length(ids), j=1:i-1
        add_edge!(socialnet, juliandict[ids[i]], juliandict[ids[j]])
    end
end

njulians = nv(socialnet)
nconnections = ne(socialnet)

info("$njulians Julians and $nconnections connections")

INFO: 1500 Julians and 41090 connections


For each node of the social network, we use GitHub API to retrieve user information:

In [11]:
# HTTP requests on https://api.github.com
juliansinfo = []
@showprogress 1 "Retrieving Julian info..." for julian in julians
    resp = HTTP.get("https://api.github.com/users/$julian?access_token=$mytoken")
    htmlbody = identity(String(resp.body))
    push!(juliansinfo, JSON.Parser.parse(htmlbody))
end

Retrieving Julian info...100% Time: 0:04:59


If the user has typed an address on his profile, we find an approximate latitude/longitude with Google Maps geocoding API:

In [12]:
locnames = []
latitudes = []
longitudes = []
countries = []
@showprogress 1 "Geocoding Julian address..." for julian in juliansinfo
    address = julian["location"]
    if address ≠ nothing
        address = replace(address, "–", "")
        address = replace(address, " ", "+")
        resp = HTTP.get("http://maps.google.com/maps/api/geocode/json?address=$address")
        htmlbody = identity(String(resp.body))
        results = JSON.Parser.parse(htmlbody)["results"]
        if length(results) > 0
            geoinfo = results[1]
            locname = geoinfo["formatted_address"]
            loccoords = geoinfo["geometry"]["location"]
            push!(locnames, locname)
            push!(latitudes, loccoords["lat"])
            push!(longitudes, loccoords["lng"])
            for comp in geoinfo["address_components"]
                if "country" ∈ comp["types"]
                    push!(countries, comp["long_name"])
                end
            end
        else
            push!(locnames, nothing)
            push!(latitudes, nothing)
            push!(longitudes, nothing)
            push!(countries, nothing)
        end
    else
        push!(locnames, nothing)
        push!(latitudes, nothing)
        push!(longitudes, nothing)
        push!(countries, nothing)
    end
end

Geocoding Julian address...100% Time: 0:03:36


Finally, we use JSON again to save the data:

In [13]:
# construct JSON
usernodes = [Dict("id"=>julian["login"],
                  "name"=>julian["name"],
                  "avatar_url"=>julian["avatar_url"],
                  "contributions"=>contribdict[julian["login"]],
                  "location"=>locnames[i],
                  "latitude"=>latitudes[i],
                  "longitude"=>longitudes[i],
                  "country"=>countries[i]) for (i,julian) in enumerate(juliansinfo)]

userlinks = [Dict("source"=>julians[u], "target"=>julians[v]) for (u,v) in edges(socialnet)]

userdata = Dict("nodes"=>usernodes, "links"=>userlinks)

# write to file
open("Julians.json", "w") do f
  JSON.print(f, userdata, 2)
end